In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as rn
import os

In [2]:


set_seed = 46
np.random.seed(set_seed)
rn.seed(set_seed)
tf.random.set_seed(set_seed)

In [ ]:
import os
import pandas as pd


file_path = 'fer2013new.csv'
df = pd.read_csv(file_path)


df_cleaned = df.drop(columns=['NF'])

df_cleaned = df_cleaned.dropna(subset=['Image name'])


df_cleaned['Image name'] = df_cleaned['Image name'].astype(str)


df_cleaned = df_cleaned[df_cleaned['Image name'].apply(lambda x: os.path.exists(os.path.join("allimage", x)))]


df = df_cleaned
print(df.columns)
print(df.shape)


LABELS = list(df.columns[2:])
print("LABELS:", LABELS)
print("All classes:", len(LABELS))

label_train = df[df['Usage'] == 'Training']
label_valid = df[df['Usage'] == 'PublicTest']
label_test = df[df['Usage'] == 'PrivateTest']


TRAIN_DATA_SIZE = label_train.shape[0]
VALID_DATA_SIZE = label_valid.shape[0]
TEST_DATA_SIZE = label_test.shape[0]


print("TRAIN_DATA_SIZE:", TRAIN_DATA_SIZE)
print("VALID_DATA_SIZE:", VALID_DATA_SIZE)
print("TEST_DATA_SIZE:", TEST_DATA_SIZE)


In [ ]:

image_folder = "allimage"


def preprocess_image(image_name, label):
    image_path = tf.strings.join([image_folder, image_name], separator="/")
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=1)  
    image = tf.image.resize(image, [224, 224]) 
    image = tf.image.grayscale_to_rgb(image) 
    image = image / 255.0  
    return image, label

def create_dataset(dataframe, batch_size=32, shuffle=True):
    image_paths = dataframe['Image name'].values
    labels = dataframe[LABELS].values
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset


batch_size = 32
train_ds = create_dataset(label_train, batch_size=batch_size, shuffle=True)
val_ds = create_dataset(label_valid, batch_size=batch_size, shuffle=False)
test_ds = create_dataset(label_test, batch_size=batch_size, shuffle=False)


for images, labels in train_ds.take(1):
    print(f"Batch image shape: {images.shape}, Batch label shape: {labels.shape}")

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt


set_seed = 46
np.random.seed(set_seed)
tf.random.set_seed(set_seed)


file_path = 'fer2013new.csv'  
image_folder = "allimage"      


df = pd.read_csv(file_path)


df_cleaned = df.drop(columns=['NF'], errors='ignore')  
df_cleaned = df_cleaned.dropna(subset=['Image name'])  
df_cleaned['Image name'] = df_cleaned['Image name'].astype(str)  
df_cleaned = df_cleaned[df_cleaned['Image name'].apply(lambda x: os.path.exists(os.path.join(image_folder, x)))]   


LABELS = list(df_cleaned.columns[2:]) 
print("Labels:", LABELS)




for label in LABELS:
    df_cleaned[label] = df_cleaned[label].apply(lambda x: 1 if x > 0 else 0)




label_train = df_cleaned[df_cleaned['Usage'] == 'Training']
label_valid = df_cleaned[df_cleaned['Usage'] == 'PublicTest']
label_test = df_cleaned[df_cleaned['Usage'] == 'PrivateTest']

TRAIN_DATA_SIZE = label_train.shape[0]
VALID_DATA_SIZE = label_valid.shape[0]
TEST_DATA_SIZE = label_test.shape[0]

print(f"Train size: {TRAIN_DATA_SIZE}, Validation size: {VALID_DATA_SIZE}, Test size: {TEST_DATA_SIZE}")


def preprocess_image(image_name, label):
    image_path = tf.strings.join([image_folder, image_name], separator="/")
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=1) 
    image = tf.image.resize(image, [64, 64]) 
    image = tf.image.grayscale_to_rgb(image)  
    image = image / 255.0 
    return image, label


def create_dataset(dataframe, batch_size=32, shuffle=True):
    image_paths = dataframe['Image name'].values
    labels = dataframe[LABELS].values
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size) 
    dataset = dataset.repeat() 
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)  
    return dataset


batch_size = 32
train_ds = create_dataset(label_train, batch_size=batch_size, shuffle=True)
val_ds = create_dataset(label_valid, batch_size=batch_size, shuffle=False)
test_ds = create_dataset(label_test, batch_size=batch_size, shuffle=False)


for images, labels in train_ds.take(1):
    print(f"Batch image shape: {images.shape}, Batch label shape: {labels.shape}")


initializer = keras.initializers.GlorotUniform(seed=set_seed)
number_of_classes = len(LABELS)

base_model = keras.applications.ResNet50(
    weights='imagenet',          
    input_shape=(64, 64, 3),   
    include_top=False,          
    pooling='avg'                
)

base_model.trainable = True  
fine_tune_at = 120
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False  

inputs = keras.Input(shape=(64, 64, 3))
x = base_model(inputs)
x = keras.layers.Dense(number_of_classes, kernel_initializer=initializer)(x)
x = keras.layers.Dropout(0.5, seed=set_seed)(x)
outputs = keras.layers.Dense(number_of_classes, kernel_initializer=initializer, activation='sigmoid')(x)

model = keras.Model(inputs, outputs)



model.summary()


lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-5,
    decay_steps=TRAIN_DATA_SIZE // batch_size * 10,
    decay_rate=0.95
)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)


es_callback = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)


history = model.fit(
    train_ds,
    steps_per_epoch=TRAIN_DATA_SIZE // batch_size,
    validation_data=val_ds,
    validation_steps=VALID_DATA_SIZE // batch_size,
    epochs=20,
    callbacks=[es_callback]
)


loss, accuracy = model.evaluate(test_ds, steps=TEST_DATA_SIZE // batch_size)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


